In [5]:
from web3 import Web3
import pandas as pd
import datetime
from concurrent.futures import ProcessPoolExecutor
import numpy as np

In [7]:
key_1 = "4d08ab7e29294be397e15ec5503401ca"
key_2 = "d4d8ddd18ef6438ea49fd3d5006b7eba"
key_3 = "b7b06ad6a7304e2197efa10b79e1c867"
key_4 = "58bce48fe80b451c96fee8ef35bbcce8"

provider = "https://mainnet.infura.io/v3/"

In [48]:
w3 = Web3(Web3.HTTPProvider(provider))

start_block = 18935888
end_block   = 18993488

transactions_data = []

for block_number in range(start_block, end_block + 1):
    block = w3.eth.get_block(block_number, full_transactions=True)
    block_timestamp = datetime.datetime.utcfromtimestamp(block.timestamp)
    
    for tx in block.transactions:
        tx_receipt = w3.eth.get_transaction_receipt(tx.hash)
        
        transactions_data.append({
            'block_number': block_number,
            'tx_hash': tx.hash.hex(),
            'gas_price': tx.gasPrice,
            'gas_used': tx_receipt.gasUsed,
            'timestamp': block_timestamp
        })

print("Dataframe saved. Number of transactions:", len(df_transactions))

df_transactions = pd.DataFrame(transactions_data)

df_transactions['gas_price_gwei'] = df_transactions['gas_price'].apply(lambda x: w3.from_wei(x, 'gwei')).astype(int)
df_transactions['tx_hash']        = df_transactions['tx_hash'].astype(str)
df_transactions['timestamp']      = pd.to_datetime(df_transactions['timestamp'])
df_transactions = df_transactions.set_index('timestamp').sort_index()

df_transactions.to_pickle('ethereum_transactions.pkl')


KeyboardInterrupt



In [ ]:
df_transactions

In [1]:
from web3 import Web3
import pandas as pd
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
import numpy as np
from pathos.multiprocessing import ProcessingPool as Pool
import multiprocessing as mp


def fetch_transactions(block_numbers, key):
    local_w3 = Web3(Web3.HTTPProvider("https://mainnet.infura.io/v3/" + key))
    transactions_data = []
    for block_number in block_numbers:
        block = local_w3.eth.get_block(block_number, full_transactions=True)
        block_timestamp = datetime.utcfromtimestamp(block.timestamp)
        for tx in block.transactions:
            tx_receipt = local_w3.eth.get_transaction_receipt(tx.hash)
            transactions_data.append({
                'block_number': block_number,
                'tx_hash': tx.hash.hex(),
                'gas_price': tx.gasPrice,
                'gas_used': tx_receipt.gasUsed,
                'timestamp': block_timestamp
            })
    return transactions_data


def main(start_block, end_block, keys):
    num_jobs = len(keys)
    block_range = list(range(start_block, end_block + 1))
    split_block_ranges = np.array_split(block_range, num_jobs)
    all_transactions = []

    with Pool(num_jobs) as pool:
        results = pool.map(lambda params: fetch_transactions(*params), 
                           [(block_numbers.tolist(), keys[i]) for i, block_numbers in enumerate(split_block_ranges)])
        for result in results:
            all_transactions.extend(result)
    
    df_transactions = pd.DataFrame(all_transactions)
    df_transactions['gas_price_gwei'] = df_transactions['gas_price'].apply(lambda x: Web3.fromWei(x, 'gwei')).astype(int)
    df_transactions['tx_hash'] = df_transactions['tx_hash'].astype(str)
    df_transactions['timestamp'] = pd.to_datetime(df_transactions['timestamp'])
    df_transactions = df_transactions.set_index('timestamp').sort_index()

    df_transactions.to_pickle('ethereum_transactions.pkl')
    print(f"Dataframe saved. Number of transactions: {len(df_transactions)}")

if __name__ == '__main__':
    mp.set_start_method('spawn')
    start_block = 18935888
    end_block   = 18993488
    provider_keys = ["4d08ab7e29294be397e15ec5503401ca",
                     "d4d8ddd18ef6438ea49fd3d5006b7eba",
                     "b7b06ad6a7304e2197efa10b79e1c867",
                     "58bce48fe80b451c96fee8ef35bbcce8"]
    main(start_block, end_block, provider_keys)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


The process has forked and you cannot use this CoreFoundation functionality safely. You MUST exec().
Break on __THE_PROCESS_HAS_FORKED_AND_YOU_CANNOT_USE_THIS_COREFOUNDATION_FUNCTIONALITY___YOU_MUST_EXEC__() to debug.
The process has forked and you cannot use this CoreFoundation functionality safely. You MUST exec().
Break on __THE_PROCESS_HAS_FORKED_AND_YOU_CANNOT_USE_THIS_COREFOUNDATION_FUNCTIONALITY___YOU_MUST_EXEC__() to debug.
The process has forked and you cannot use this CoreFoundation functionality safely. You MUST exec().
Break on __THE_PROCESS_HAS_FORKED_AND_YOU_CANNOT_USE_THIS_COREFOUNDATION_FUNCTIONALITY___YOU_MUST_EXEC__() to debug.
The process has forked and you cannot use this CoreFoundation functionality safely. You MUST exec().
Break on __THE_PROCESS_HAS_FORKED_AND_YOU_CANNOT_USE_THIS_COREFOUNDATION_FUNCTIONALITY___YOU_MUST_EXEC__() to debug.
The process has forked and you cannot use this CoreFoundation functionality safely. You MUST exec().
Break on __THE_PROCESS_HAS_

KeyboardInterrupt: 